In [289]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [357]:
#pull dataframe
csv_path = tf.keras.utils.get_file(
    origin='https://drive.google.com/uc?export=download&id=1cqwR4NLFlgZxNL7JZB5uscDthV7HlUxb',
    fname='processed_combined_1') #this is two different csv files combined together
df = pd.read_csv(csv_path)

X = df.loc[:, ['1.0 std','2.5 std', '10 std']] #dataframe with only these three columns, this is what the model is currently trained off of
y = df.loc[:, ['Fire']] #dataframe for output

X, y = X.to_numpy(), y.to_numpy() #convert to numpy arrays

y = y.flatten() #reshaping each
X = X.reshape((X.shape[0], X.shape[1], 1))


X_train, y_train = X[:3250], y[:3250]
X_val, y_val = X[3250:4000], y[3250:4000]
X_test, y_test = X[4000:], y[4000:]


X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((3250, 3, 1), (3250,), (750, 3, 1), (750,), (1057, 3, 1), (1057,))

In [365]:
model = keras.Sequential()
model.add(keras.Input(shape=(3, 1))) #no specific amount of timesteps, 3 pixels in each row/timestep
model.add(layers.LSTM(512, return_sequences=True, activation='tanh')) #returns the output of each timestep so we can stack multiple RNN layers
model.add(layers.LSTM(256, activation='tanh')) #no return sequences so that normal dense layers can proceed it
model.add(layers.Dense(2))

#could do LSTM, GRU, or RNN and can change by literally just swapping them out

print(model.summary())

#RNNs typically use tanh instead of relu

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), #fromlogits is true since we don't have softmax activation in final dense layer
    optimizer=keras.optimizers.Adam(learning_rate=0.001), #dont make learning rate too high or will result in overfitment of training data
    metrics=['accuracy']
)

model.fit(X_train, y_train, batch_size=32, validation_data=(X_val, y_val),epochs=10, verbose=1)

model.evaluate(X_test, y_test, batch_size=32, verbose=1)

Model: "sequential_31"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_39 (LSTM)                  │ (None, 3, 512)         │     1,052,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_40 (LSTM)                  │ (None, 256)            │       787,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,840,642 (7.02 MB)

 Trainable params: 1,840,642 (7.02 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8859 - loss: 0.3880 - val_accuracy: 0.8587 - val_loss: 0.3074
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8968 - loss: 0.2455 - val_accuracy: 0.8187 - val_loss: 0.3517
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8907 - loss: 0.2487 - val_accuracy: 0.8680 - val_loss: 0.3503
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8872 - loss: 0.2539 - val_accuracy: 0.8800 - val_loss: 0.3097
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9123 - loss: 0.2205 - val_accuracy: 0.8347 - val_loss: 0.3134
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9174 - loss: 0.2075 - val_accuracy: 0.8173 - val_loss: 0.3226
Epoch 7/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9094 - loss: 0.2158 - val_accuracy: 0.8173 - val_loss: 0.3335
Epoch 8/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9221 - loss: 0.1951 - val_accura

[0.39720064401626587, 0.8770104050636292]